In [1]:
import os
%pwd

'd:\\end-to-end-ml-project-with-mlflow\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\end-to-end-ml-project-with-mlflow'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig():
    root_dir:Path
    data_path:Path

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [6]:
from mlProject import logger
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
class DataTransformation():
    def __init__(self,config:DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        
        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data,test_size=0.2,random_state=42)

        num_pipeline= Pipeline(
            steps=[
            ("imputer",SimpleImputer(strategy="median")),
            ("scaler",StandardScaler())

            ]
        )

        target_column_name="quality"
        numerical_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
            'pH', 'sulphates', 'alcohol', 'quality']

        input_feature_train_df=train.drop(columns=[target_column_name],axis=1)
        target_feature_train_df=train[target_column_name]

        input_feature_test_df=test.drop(columns=[target_column_name],axis=1)
        target_feature_test_df=test[target_column_name]

        logger.info(
            f"Applying preprocessing object on training dataframe and testing dataframe."
        )

        input_feature_train_arr=num_pipeline.fit_transform(input_feature_train_df)
        input_feature_test_arr=num_pipeline.transform(input_feature_test_df)

        train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
        test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index=False,header=True)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index=False,header=True)

        logger.info("Splitted the data into test & training data")
        logger.info(f"Shape of the training data is : {train.shape}")
        logger.info(f"Shape of the test data is : {test.shape}")

        print(train.shape)
        print(test.shape)

        return(
            train_arr,
            test_arr,
        )

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2023-12-04 14:26:59,894: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-04 14:26:59,902: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-04 14:26:59,902: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-04 14:26:59,902: INFO: common: created directory at: artifacts]
[2023-12-04 14:26:59,902: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-04 14:26:59,929: INFO: 3019443674: Applying preprocessing object on training dataframe and testing dataframe.]
[2023-12-04 14:26:59,959: INFO: 3019443674: Splitted the data into test & training data]
[2023-12-04 14:26:59,973: INFO: 3019443674: Shape of the training data is : (1279, 12)]
[2023-12-04 14:26:59,985: INFO: 3019443674: Shape of the test data is : (320, 12)]
(1279, 12)
(320, 12)
